# Complete Reinforcement Learning Project: Bandit Problem and Gridworld


This notebook demonstrates reinforcement learning techniques applied to two classic problems:
1. **Bandit Problem**: Solving a 10-armed bandit problem with various exploration-exploitation strategies.
2. **Gridworld**: Using dynamic programming to find optimal policies in a grid environment.


## 1. Bandit Problem

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import random

class Bandit:
    def __init__(self, true_reward=0):
        self.q_true = np.random.normal(true_reward, 1, 10)
        self.q_estimate = np.zeros(10)
        self.action_count = np.zeros(10)

    def pull(self, action):
        return np.random.normal(self.q_true[action], 1)

    def update_estimates(self, action, reward):
        self.action_count[action] += 1
        self.q_estimate[action] += (1 / self.action_count[action]) * (reward - self.q_estimate[action])

def epsilon_greedy(bandit, epsilon, n_pulls=1000):
    rewards = []
    for _ in range(n_pulls):
        action = np.random.randint(10) if random.random() < epsilon else np.argmax(bandit.q_estimate)
        reward = bandit.pull(action)
        bandit.update_estimates(action, reward)
        rewards.append(reward)
    return np.cumsum(rewards) / (np.arange(n_pulls) + 1)

def optimistic_initial_values(bandit, initial_value, n_pulls=1000):
    bandit.q_estimate = np.ones(10) * initial_value
    rewards = []
    for _ in range(n_pulls):
        action = np.argmax(bandit.q_estimate)
        reward = bandit.pull(action)
        bandit.update_estimates(action, reward)
        rewards.append(reward)
    return np.cumsum(rewards) / (np.arange(n_pulls) + 1)

def ucb(bandit, c, n_pulls=1000):
    rewards = []
    for t in range(1, n_pulls + 1):
        ucb_estimates = bandit.q_estimate + c * np.sqrt(np.log(t) / (bandit.action_count + 1e-5))
        action = np.argmax(ucb_estimates)
        reward = bandit.pull(action)
        bandit.update_estimates(action, reward)
        rewards.append(reward)
    return np.cumsum(rewards) / (np.arange(n_pulls) + 1)

def gradient_bandit(bandit, alpha, baseline=True, n_pulls=1000):
    preferences = np.zeros(10)
    rewards = []
    average_reward = 0
    for t in range(n_pulls):
        exp_preferences = np.exp(preferences)
        action_probs = exp_preferences / np.sum(exp_preferences)
        action = np.random.choice(10, p=action_probs)
        reward = bandit.pull(action)
        rewards.append(reward)
        
        if baseline:
            average_reward += (reward - average_reward) / (t + 1)
        
        for a in range(10):
            if a == action:
                preferences[a] += alpha * (reward - (average_reward if baseline else 0)) * (1 - action_probs[a])
            else:
                preferences[a] -= alpha * (reward - (average_reward if baseline else 0)) * action_probs[a]
    return np.cumsum(rewards) / (np.arange(n_pulls) + 1)


### Bandit Problem Results

In [ ]:

# ε-Greedy Results
epsilons = [0, 0.1, 0.01]
rewards_epsilon_greedy = [epsilon_greedy(Bandit(), epsilon) for epsilon in epsilons]
for i, reward in enumerate(rewards_epsilon_greedy):
    plt.plot(reward, label=f'ε = {epsilons[i]}')
plt.xlabel("Steps")
plt.ylabel("Average Reward")
plt.legend()
plt.title("ε-Greedy Results")
plt.show()

# Optimistic Initial Values
rewards_optimistic = optimistic_initial_values(Bandit(), initial_value=5)
plt.plot(rewards_optimistic, label="Optimistic Initial Value = 5")
plt.xlabel("Steps")
plt.ylabel("Average Reward")
plt.legend()
plt.title("Optimistic Initial Values Results")
plt.show()

# UCB Results
c_values = [0.1, 1, 2]
rewards_ucb = [ucb(Bandit(), c) for c in c_values]
for i, reward in enumerate(rewards_ucb):
    plt.plot(reward, label=f'c = {c_values[i]}')
plt.xlabel("Steps")
plt.ylabel("Average Reward")
plt.legend()
plt.title("UCB Results")
plt.show()

# Gradient Bandit Results
alphas = [0.1, 0.4]
rewards_gradient_bandit = [gradient_bandit(Bandit(), alpha) for alpha in alphas]
for i, reward in enumerate(rewards_gradient_bandit):
    plt.plot(reward, label=f'alpha = {alphas[i]}')
plt.xlabel("Steps")
plt.ylabel("Average Reward")
plt.legend()
plt.title("Gradient Bandit Results")
plt.show()


## 2. Gridworld

In [ ]:

class Gridworld:
    def __init__(self, size=5, gamma=1.0, theta=0.01):
        self.size = size
        self.gamma = gamma
        self.theta = theta
        self.state_values = np.zeros((size, size))
        self.policy = np.full((size, size), 'up')

    def policy_evaluation(self):
        delta = self.theta
        while delta >= self.theta:
            delta = 0
            for i in range(self.size):
                for j in range(self.size):
                    v = self.state_values[i, j]
                    new_value = self.calculate_value(i, j)
                    delta = max(delta, abs(new_value - v))
                    self.state_values[i, j] = new_value

    def calculate_value(self, i, j):
        reward = -1
        value = 0
        transitions = self.get_transitions(i, j)
        for (new_i, new_j) in transitions:
            value += 0.25 * (reward + self.gamma * self.state_values[new_i, new_j])
        return value

    def get_transitions(self, i, j):
        return [
            (max(0, i - 1), j),
            (min(self.size - 1, i + 1), j),
            (i, max(0, j - 1)),
            (i, min(self.size - 1, j + 1))
        ]

    def policy_improvement(self):
        policy_stable = True
        for i in range(self.size):
            for j in range(self.size):
                old_action = self.policy[i, j]
                new_action = self.best_action(i, j)
                self.policy[i, j] = new_action
                if old_action != new_action:
                    policy_stable = False
        return policy_stable

    def best_action(self, i, j):
        actions = ['up', 'down', 'left', 'right']
        values = [self.calculate_action_value(i, j, a) for a in actions]
        return actions[np.argmax(values)]

    def calculate_action_value(self, i, j, action):
        reward = -1
        new_i, new_j = self.get_new_position(i, j, action)
        return reward + self.gamma * self.state_values[new_i, new_j]

    def get_new_position(self, i, j, action):
        if action == 'up':
            return max(0, i - 1), j
        elif action == 'down':
            return min(self.size - 1, i + 1), j
        elif action == 'left':
            return i, max(0, j - 1)
        else:
            return i, min(self.size - 1, j + 1)


In [ ]:

# Initialize gridworld and evaluate policy
grid = Gridworld()
grid.policy_evaluation()

# Visualize state values
plt.imshow(grid.state_values, cmap='viridis')
plt.colorbar()
plt.title("State Values after Policy Evaluation")
plt.show()
